In [11]:
library('tidyverse')
library('httr')
library('jsonlite')

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()



In [1]:
# check also https://www.programmableweb.com/news/how-to-access-any-restful-api-using-r-language/how-to/2017/07/21?page=2

In [7]:

r = GET("https://www.googleapis.com/books/v1/volumes?q=python")
data = content(r, as="parsed")
print(names(data))
print(data$items[[1]])

[1] "kind"       "totalItems" "items"     
$kind
[1] "books#volume"

$id
[1] "wqeVv09Y6hIC"

$etag
[1] "X93TTSzbXvk"

$selfLink
[1] "https://www.googleapis.com/books/v1/volumes/wqeVv09Y6hIC"

$volumeInfo
$volumeInfo$title
[1] "Python"

$volumeInfo$subtitle
[1] "A Study of Delphic Myth and Its Origins"

$volumeInfo$authors
$volumeInfo$authors[[1]]
[1] "Joseph Eddy Fontenrose"


$volumeInfo$publisher
[1] "Univ of California Press"

$volumeInfo$publishedDate
[1] "1959"

$volumeInfo$description
[1] "A study of Delphic myths and their origins."

$volumeInfo$industryIdentifiers
$volumeInfo$industryIdentifiers[[1]]
$volumeInfo$industryIdentifiers[[1]]$type
[1] "ISBN_10"

$volumeInfo$industryIdentifiers[[1]]$identifier
[1] "0520040910"


$volumeInfo$industryIdentifiers[[2]]
$volumeInfo$industryIdentifiers[[2]]$type
[1] "ISBN_13"

$volumeInfo$industryIdentifiers[[2]]$identifier
[1] "9780520040915"



$volumeInfo$readingModes
$volumeInfo$readingModes$text
[1] FALSE

$volumeInfo$readingModes$imag

In [13]:
r_text <- content(r, "text")
data_json <- jsonlite::fromJSON(r_text, flatten = TRUE)
d = as.data.frame(data_json)
head(d)

,kind,totalItems,items.kind,items.id,items.etag,items.selfLink,items.volumeInfo.title,items.volumeInfo.subtitle,items.volumeInfo.authors,items.volumeInfo.publisher,⋯,items.accessInfo.embeddable,items.accessInfo.publicDomain,items.accessInfo.textToSpeechPermission,items.accessInfo.webReaderLink,items.accessInfo.accessViewStatus,items.accessInfo.quoteSharingAllowed,items.accessInfo.epub.isAvailable,items.accessInfo.pdf.isAvailable,items.accessInfo.pdf.acsTokenLink,items.searchInfo.textSnippet
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<list>,<chr>,⋯,<lgl>,<lgl>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<chr>,<chr>
1,books#volumes,441,books#volume,wqeVv09Y6hIC,X93TTSzbXvk,https://www.googleapis.com/books/v1/volumes/wqeVv09Y6hIC,Python,A Study of Delphic Myth and Its Origins,Joseph Eddy Fontenrose,Univ of California Press,⋯,TRUE,FALSE,ALLOWED,http://play.google.com/books/reader?id=wqeVv09Y6hIC&hl=&printsec=frontcover&source=gbs_api,SAMPLE,FALSE,FALSE,FALSE,NA,A study of Delphic myths and their origins.
2,books#volumes,441,books#volume,aJQILlLxRmAC,1x9G8zJhzJU,https://www.googleapis.com/books/v1/volumes/aJQILlLxRmAC,Python Programming,An Introduction to Computer Science,John M. Zelle,"Franklin, Beedle & Associates, Inc.",⋯,TRUE,FALSE,ALLOWED,http://play.google.com/books/reader?id=aJQILlLxRmAC&hl=&printsec=frontcover&source=gbs_api,SAMPLE,FALSE,FALSE,FALSE,NA,"This book is suitable for use in a university-level first course in computing (CS1), as well as the increasingly popular course known as CS0."
3,books#volumes,441,books#volume,6GzuBgAAQBAJ,2b3xQ+Fqzjc,https://www.googleapis.com/books/v1/volumes/6GzuBgAAQBAJ,Leer programmeren in Python,Simpel zelf programma's maken ook voor de Raspberry Pi,NULL,Reshift Digital,⋯,TRUE,FALSE,ALLOWED,http://play.google.com/books/reader?id=6GzuBgAAQBAJ&hl=&printsec=frontcover&source=gbs_api,SAMPLE,FALSE,FALSE,TRUE,http://books.google.nl/books/download/Leer_programmeren_in_Python-sample-pdf.acsm?id=6GzuBgAAQBAJ&format=pdf&output=acs4_fulfillment_token&dl_type=sample&source=gbs_api,"<b>Python</b> is een hoge programmeertaal die in de jaren tachtig werd ontwikkeld <br> door de Nederlander Guido van Rossum. En we moeten je teleurstellen: de <br> naam is niet afkomstig van de slagensoort <b>python</b>, maar de taal is vernoemd naar <br> Monty&nbsp;..."
4,books#volumes,441,books#volume,mCrrCAAAQBAJ,qJsyOcskT38,https://www.googleapis.com/books/v1/volumes/mCrrCAAAQBAJ,Python Scripting for Computational Science,NA,Hans Petter Langtangen,Springer Science & Business Media,⋯,TRUE,FALSE,ALLOWED_FOR_ACCESSIBILITY,http://play.google.com/books/reader?id=mCrrCAAAQBAJ&hl=&printsec=frontcover&source=gbs_api,SAMPLE,FALSE,FALSE,TRUE,http://books.google.nl/books/download/Python_Scripting_for_Computational_Scien-sample-pdf.acsm?id=mCrrCAAAQBAJ&format=pdf&output=acs4_fulfillment_token&dl_type=sample&source=gbs_api,Scripting with Python makes you productive and increases the reliability of your scientific work.
5,books#volumes,441,books#volume,carqdIdfVlYC,0vcIQokQKzc,https://www.googleapis.com/books/v1/volumes/carqdIdfVlYC,Python,NA,Chris Fehily,Peachpit Press,⋯,TRUE,FALSE,ALLOWED_FOR_ACCESSIBILITY,http://play.google.com/books/reader?id=carqdIdfVlYC&hl=&printsec=frontcover&source=gbs_api,SAMPLE,FALSE,FALSE,FALSE,NA,"Using Mathematical Functions <b>Python</b> includes functions that perform <br> mathematical calculations . Table 3.7 lists the popular ones ; others are <br> described in Section 2.3 , “ Built - in Functions , &quot; of the <b>Python</b> Library Reference <br> . Figure 3.10 shows&nbsp;..."
6,books#volumes,441,books#volume,fzUCGtyg0MMC,RUkIPx/b4Wg,https://www.googleapis.com/books/v1/volumes/fzUCGtyg0MMC,Python Programming On Win32,Help for Windows Programmers,"Mark Hammond , Andy Robinson","""O'Reilly Media, Inc.""",⋯,TRUE,FALSE,ALLOWED,http://play.google.com/books/reader?id=fzUCGtyg0MMC&hl=&printsec=frontcover&source=gbs_api,SAMPLE,FALSE,FALSE,TRUE,NA,"Appendixes Key <b>Python</b> Modules and Functions The <b

In [15]:
# smaller table to use as output for in book
d  %>% select(kind:items.etag)  %>% head()

,kind,totalItems,items.kind,items.id,items.etag
,<chr>,<int>,<chr>,<chr>,<chr>
1,books#volumes,441,books#volume,wqeVv09Y6hIC,X93TTSzbXvk
2,books#volumes,441,books#volume,aJQILlLxRmAC,1x9G8zJhzJU
3,books#volumes,441,books#volume,6GzuBgAAQBAJ,2b3xQ+Fqzjc
4,books#volumes,441,books#volume,mCrrCAAAQBAJ,qJsyOcskT38
5,books#volumes,441,books#volume,carqdIdfVlYC,0vcIQokQKzc
6,books#volumes,441,books#volume,fzUCGtyg0MMC,RUkIPx/b4Wg


In [16]:
require('httr')
require('jsonlite')
require(glue)
library(data.table)

i = 0
j = 1

alldata = list()
while (TRUE) {
    r = GET(glue("https://www.googleapis.com/books/v1/volumes?q=python&maxResults=40&startIndex={i}"))
    r_text <- content(r, "text")
    data_json <- jsonlite::fromJSON(r_text, flatten = TRUE)
    if (length(data_json$items)==0) {break}
    alldata[[j]] = as.data.frame(data_json)
    i = i + 40
    j = j + 1} 

d = rbindlist(alldata, fill=TRUE)

Loading required package: glue


Attaching package: ‘glue’


The following object is masked from ‘package:dplyr’:

    collapse



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




In [17]:
# smaller table to use as output for in book
d  %>% select(kind:items.etag)  %>% head()

kind,totalItems,items.kind,items.id,items.etag,items.selfLink,items.volumeInfo.title,items.volumeInfo.subtitle,items.volumeInfo.authors,items.volumeInfo.publisher,⋯,items.accessInfo.publicDomain,items.accessInfo.textToSpeechPermission,items.accessInfo.webReaderLink,items.accessInfo.accessViewStatus,items.accessInfo.quoteSharingAllowed,items.accessInfo.epub.isAvailable,items.accessInfo.epub.acsTokenLink,items.accessInfo.pdf.isAvailable,items.accessInfo.pdf.acsTokenLink,items.searchInfo.textSnippet
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<list>,<chr>,⋯,<lgl>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<lgl>,<chr>,<chr>
books#volumes,343,books#volume,h56ansk4SyQC,T4HfpbdDshc,https://www.googleapis.com/books/v1/volumes/h56ansk4SyQC,Python,A Study of Delphic Myth and Its Origins,Joseph Eddy Fontenrose,Biblo & Tannen Publishers,⋯,FALSE,ALLOWED,http://play.google.com/books/reader?id=h56ansk4SyQC&hl=&printsec=frontcover&source=gbs_api,SAMPLE,FALSE,FALSE,NA,FALSE,NA,NA
books#volumes,343,books#volume,aJQILlLxRmAC,1x9G8zJhzJU,https://www.googleapis.com/books/v1/volumes/aJQILlLxRmAC,Python Programming,An Introduction to Computer Science,John M. Zelle,"Franklin, Beedle & Associates, Inc.",⋯,FALSE,ALLOWED,http://play.google.com/books/reader?id=aJQILlLxRmAC&hl=&printsec=frontcover&source=gbs_api,SAMPLE,FALSE,FALSE,NA,FALSE,NA,"This book is suitable for use in a university-level first course in computing (CS1), as well as the increasingly popular course known as CS0."
books#volumes,343,books#volume,6GzuBgAAQBAJ,2b3xQ+Fqzjc,https://www.googleapis.com/books/v1/volumes/6GzuBgAAQBAJ,Leer programmeren in Python,Simpel zelf programma's maken ook voor de Raspberry Pi,NULL,Reshift Digital,⋯,FALSE,ALLOWED,http://play.google.com/books/reader?id=6GzuBgAAQBAJ&hl=&printsec=frontcover&source=gbs_api,SAMPLE,FALSE,FALSE,NA,TRUE,http://books.google.nl/books/download/Leer_programmeren_in_Python-sample-pdf.acsm?id=6GzuBgAAQBAJ&format=pdf&output=acs4_fulfillment_token&dl_type=sample&source=gbs_api,"<b>Python</b> is een hoge programmeertaal die in de jaren tachtig werd ontwikkeld <br> door de Nederlander Guido van Rossum. En we moeten je teleurstellen: de <br> naam is niet afkomstig van de slagensoort <b>python</b>, maar de taal is vernoemd naar <br> Monty&nbsp;..."
books#volumes,343,books#volume,mCrrCAAAQBAJ,qJsyOcskT38,https://www.googleapis.com/books/v1/volumes/mCrrCAAAQBAJ,Python Scripting for Computational Science,NA,Hans Petter Langtangen,Springer Science & Business Media,⋯,FALSE,ALLOWED_FOR_ACCESSIBILITY,http://play.google.com/books/reader?id=mCrrCAAAQBAJ&hl=&printsec=frontcover&source=gbs_api,SAMPLE,FALSE,FALSE,NA,TRUE,http://books.google.nl/books/download/Python_Scripting_for_Computational_Scien-sample-pdf.acsm?id=mCrrCAAAQBAJ&format=pdf&output=acs4_fulfillment_token&dl_type=sample&source=gbs_api,Scripting with Python makes you productive and increases the reliability of your scientific work.
books#volumes,343,books#volume,carqdIdfVlYC,0vcIQokQKzc,https://www.googleapis.com/books/v1/volumes/carqdIdfVlYC,Python,NA,Chris Fehily,Peachpit Press,⋯,FALSE,ALLOWED_FOR_ACCESSIBILITY,http://play.google.com/books/reader?id=carqdIdfVlYC&hl=&printsec=frontcover&source=gbs_api,SAMPLE,FALSE,FALSE,NA,FALSE,NA,"Using Mathematical Functions <b>Python</b> includes functions that perform <br> mathematical calculations . Table 3.7 lists the popular ones ; others are <br> described in Section 2.3 , “ Built - in Functions , &quot; of the <b>Python</b> Library Reference <br> . Figure 3.10 shows&nbsp;..."
books#volumes,343,books#volume,fzUCGtyg0MMC,RUkIPx/b4Wg,https://www.googleapis.com/books/v1/volumes/fzUCGtyg0MMC,Python Programming On Win32,Help for Windows Programmers,"Mark Hammond , Andy Robinson","""O'Reilly Media, Inc.""",⋯,FALSE,ALLOWED,http://play.google.com/books/reader?id=fzUCGtyg0MMC&hl=&printsec=frontcover&source=gbs_api,SAMPLE,FALSE,FALSE,NA,TRUE,NA,"Appendixes Key <b>Python</b> Modules and Functions The <b>Python</b> library is huge. This <br> is the Title of the 

In [ ]:
# Using twitter to get tweets and user info
lookup_users("vanatteveldt")  %>% select("user_id", "screen_name", "location")
tweets = search_tweets("#covid", n = 100, include_rts = FALSE) 
head(tweets)

In [ ]:
# Getting location data
library(tmaptools)
geocode_OSM(c("Amsterdam", "New York"))

In [ ]:
reply_users = setdiff(tweets$reply_to_screen_name, tweets$screen_name) %>% na.omit()
reply_users = lookup_users(reply_users) %>% select(screen_name, location) 
# Combine with existing users, filter blank locations and duplicate names, and clean location names
users = tweets %>% select(screen_name, location) %>%
  bind_rows(reply_users) %>% 
  unique() %>%
  filter(location != "") %>% 
  mutate(location = str_replace_all(location, "[^\\p{L} ,/-]+", " "))

# Use Open Street Map API to "geocode" the locations and merge back with users
locations = geocode_OSM(unique(users$location)) %>% 
  select(location=query, lat, long=lon)
users = users %>% left_join(locations)
head(users)